In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE
import joblib


## Load dataset 

In [ ]:
df = pd.read_csv('../data/processed/CreditCard_Processed.csv')
print(df.shape)
print(df['Class'].value_counts())


## Split Features & Target

In [ ]:
X = df.drop(columns=['Class'])
y = df['Class']


## Handle Imbalance

In [ ]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)
print("Before SMOTE:", y.value_counts(normalize=True))
print("After SMOTE:", pd.Series(y_res).value_counts(normalize=True))


## Train–Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)


## Logistic Regression Baseline

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr.fit(X_train_scaled, y_train)
y_pred = lr.predict(X_test_scaled)
y_proba = lr.predict_proba(X_test_scaled)[:,1]

precision, recall, _ = precision_recall_curve(y_test, y_proba)
auc_pr = auc(recall, precision)
print(f"LogReg F1 = {f1_score(y_test,y_pred):.4f} AUC-PR = {auc_pr:.4f}")
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, fmt='d'); plt.title('LR Confusion Matrix');


## Random Forest Ensemble

In [ ]:
rf = RandomForestClassifier(n_estimators=150, max_depth=10,
                            class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

precision, recall, _ = precision_recall_curve(y_test, y_proba_rf)
auc_pr_rf = auc(recall, precision)
print(f"RF F1 = {f1_score(y_test,y_pred_rf):.4f} AUC-PR = {auc_pr_rf:.4f}")
sns.heatmap(confusion_matrix(y_test,y_pred_rf), annot=True, fmt='d'); plt.title('RF Confusion Matrix');


## Stratified K-Fold Cross-Validation

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
lr_cv = cross_val_score(lr, scaler.fit_transform(X_res), y_res, cv=skf, scoring='f1')
rf_cv = cross_val_score(rf, X_res, y_res, cv=skf, scoring='f1')

print(f"LogReg CV F1 = {lr_cv.mean():.4f} ± {lr_cv.std():.4f}")
print(f"RF CV F1 = {rf_cv.mean():.4f} ± {rf_cv.std():.4f}")


## Comparison & Save Model

In [ ]:
comparison = pd.DataFrame({
    'Model':['Logistic Regression','Random Forest'],
    'F1':[f1_score(y_test,y_pred),f1_score(y_test,y_pred_rf)],
    'AUC-PR':[auc_pr,auc_pr_rf],
    'CV Mean F1':[lr_cv.mean(),rf_cv.mean()],
    'CV Std F1':[lr_cv.std(),rf_cv.std()]
})
print(comparison)

joblib.dump(rf, '../models/random_forest_credit.pkl')
print("✅ Saved Random Forest model to models/random_forest_credit.pkl")
